In [2]:
spark

res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3e38562f


In [3]:
spark.version

res2: String = 2.3.1


# Kaggle Dataset Load

In [4]:
//val input = sc.textFile("Scorecard.csv").map(line =>line.split(","))

### Create a header index of the dataset

In [5]:
//val header = sc.parallelize(input.take(1))

In [6]:
//val headerIndex = header.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

In [7]:
//headerIndex.coalesce(1).saveAsTextFile("index")

Originally, I exported and imported the data. Not sure why.

In [8]:
// header.map(_.mkString(",")).coalesce(1).saveAsTextFile("header")

In [9]:
// val headerLoad = sc.textFile("header.csv")

In [10]:
// val headerList= headerLoad.flatMap(line => line.split(","))

In [11]:
// val headerIndex = headerList.zipWithIndex

In [12]:
//val idxSat = headerIndex.lookup("SAT_AVG")(0).toInt

# DoE Dataset Load concise

In [13]:

val in2016 = sc.textFile("MERGED2016_17_PP.csv").map(x => x + ",2016").map(line =>line.split(","))

val in2015 = sc.textFile("MERGED2015_16_PP.csv").map(x => x + ",2015").map(line =>line.split(","))

val in2014 = sc.textFile("MERGED2014_15_PP.csv").map(x => x + ",2014").map(line =>line.split(","))

val in2013 = sc.textFile("MERGED2013_14_PP.csv").map(x => x + ",2013").map(line =>line.split(","))

val in2012 = sc.textFile("MERGED2012_13_PP.csv").map(x => x + ",2012").map(line =>line.split(","))

val in2011 = sc.textFile("MERGED2011_12_PP.csv").map(x => x + ",2011").map(line =>line.split(","))

val in2010 = sc.textFile("MERGED2010_11_PP.csv").map(x => x + ",2010").map(line =>line.split(","))

val in2009 = sc.textFile("MERGED2009_10_PP.csv").map(x => x + ",2009").map(line =>line.split(","))

val in2008 = sc.textFile("MERGED2008_09_PP.csv").map(x => x + ",2008").map(line =>line.split(","))

val in2007 = sc.textFile("MERGED2007_08_PP.csv").map(x => x + ",2007").map(line =>line.split(","))


in2016: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[3] at map at <console>:25
in2015: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[7] at map at <console>:27
in2014: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[11] at map at <console>:29
in2013: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[15] at map at <console>:31
in2012: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[19] at map at <console>:33
in2011: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[23] at map at <console>:35
in2010: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[27] at map at <console>:37
in2009: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[31] at map at <console>:39
in2008: org.apache.spark.rdd.RDD[Array[String]]...

## Compile the 10 year RDD.

In [14]:
val scoreCard10Year = sc.union(in2016, in2015, in2014, in2013, in2012, in2011, in2010, in2009, in2008, in2007)
val twoYear = in2016.union(in2015)
scoreCard10Year.take(1)
twoYear.take(1)

scoreCard10Year: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[40] at union at <console>:36
twoYear: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[41] at union at <console>:37
res12: Array[Array[String]] = Array(Array(UNITID, OPEID, OPEID6, INSTNM, CITY, STABBR, ZIP, ACCREDAGENCY, INSTURL, NPCURL, SCH_DEG, HCM2, MAIN, NUMBRANCH, PREDDEG, HIGHDEG, CONTROL, ST_FIPS, REGION, LOCALE, LOCALE2, LATITUDE, LONGITUDE, CCBASIC, CCUGPROF, CCSIZSET, HBCU, PBI, ANNHI, TRIBAL, AANAPII, HSI, NANTI, MENONLY, WOMENONLY, RELAFFIL, ADM_RATE, ADM_RATE_ALL, SATVR25, SATVR75, SATMT25, SATMT75, SATWR25, SATWR75, SATVRMID, SATMTMID, SATWRMID, ACTCM25, ACTCM75, ACTEN25, ACTEN75, ACTMT25, ACTMT75, ACTWR25, ACTWR75, ACTCMMID, ACTENMID, ACTMTMID, ACTWRMID, SAT_AVG, SAT_AVG_ALL, PCIP01, PCIP03, PCIP04, PC...

### Create a header index of the dataset

In [15]:
val header2016 = sc.parallelize(in2016.take(1))

val header2016Index = header2016.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

// header2016Index.coalesce(1).saveAsTextFile("index2016") //Textfiles only made once

val header2007 = sc.parallelize(in2007.take(1))

val header2007Index = header2007.map(_.mkString(",")).flatMap(line => line.split(",")).zipWithIndex

// header2007Index.coalesce(1).saveAsTextFile("index2007") // Textfiles only made once

header2016: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[42] at parallelize at <console>:28
header2016Index: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[45] at zipWithIndex at <console>:30
header2007: org.apache.spark.rdd.RDD[Array[String]] = ParallelCollectionRDD[46] at parallelize at <console>:34
header2007Index: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[49] at zipWithIndex at <console>:36


## Count the rows of the kaggle compiled dataset

In [16]:
//input.count()

In [17]:
//input.map( x => x(1)).distinct().count()

In [18]:
// val year = input.map( x => x(1730))

// year.take(1100).foreach(println)

// It looks like there are a lot of blank lines in there early on in the dataset. 

//val notBlankYear = input.filter(x => x(1730) != "")

//notBlankYear.count()

In [19]:
//notBlankYear.map(x => x(1730)).take(20)

//notBlankYear.filter(x => x(1730) contains "2013").count()

## Count of rows for the compiled DoE Scorecard

In [20]:
scoreCard10Year.count()

res17: Long = 74181


### Number of Schools in the DoE dataset

In [21]:
scoreCard10Year.map(x=> x(1)).distinct().count()

res18: Long = 9743


# St Thomas RDD  
This RDD contains all of the St. Thomas results over the years

In [22]:
val ust = scoreCard10Year.filter(x => x(0) contains "174914")

ust: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[54] at filter at <console>:38


In [23]:
//ust.take(1)

In [24]:
ust.count()

res20: Long = 10


In [25]:
val ustHeader = header2016.union(ust)
// ustHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("ustHeader")

ustHeader: org.apache.spark.rdd.RDD[Array[String]] = UnionRDD[55] at union at <console>:42


# Create an index of the header 

Really had a tough time figuring out how to get the header loaded as just a string. Loading it from the original source results in type RDD[Array[String]], so I ended up using a CSV file that I outputed from the original input. Ended up just needing a **flatMap** to keep it in the correct format. 

# Minnesota Schools  
This rdd contains all of the Minnesota universities for a statewide comparison

In [26]:
val mnColleges = scoreCard10Year.filter( x => x(5) contains "MN").filter( x => x(1899) contains "2016")

mnColleges: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[57] at filter at <console>:38


In [27]:
val mnColLoc = mnColleges.filter(x => x(1899) contains "2016").filter(x => x(36) != "NULL").map(x => x(3) + "," + x(21) + "," + x(22) + "," + x(36))

mnColLoc: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[60] at map at <console>:40


In [28]:
// mnColLoc.take(5).foreach(println) // Only needs to run once.

Augsburg University,44.966307,-93.240436,0.4514
Bemidji State University,47.482341,-94.874119,0.6408
Bethany Lutheran College,44.166649,-93.990805,0.8049
Bethel University,45.057655,-93.161663,0.8237
Carleton College,44.462318,-93.154666,0.2262


### Create and export RDD to CSV for mapping 

In [29]:
val mnColLocHead = Array("Name,Lat,Long,AdmRate")
val mnLocRDD = sc.parallelize(mnColLocHead)
val mnColLocFin = mnLocRDD.union(mnColLoc)
//mnColLocFin.coalesce(1).saveAsTextFile("MnSchools") // only run once

mnColLocHead: Array[String] = Array(Name,Lat,Long,AdmRate)
mnLocRDD: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[61] at parallelize at <console>:43
mnColLocFin: org.apache.spark.rdd.RDD[String] = UnionRDD[62] at union at <console>:44


# Regional Schools  
This RDD contains schools from the upper-midwest (Minnesota, Wisconsin, North Dakota, South Dakota, Iowa, Nebraska). This RDD will be used for regional comparisons. 

In [30]:
val upMidWestColleges = scoreCard10Year.filter( x => x(19) contains "Plains")

upMidWestColleges: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[63] at filter at <console>:38


In [1]:
%%python

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap


//mnAdmDf = pd.read_csv("mnColLocAdm.csv")

fig = pt.figure(figsize=(8, 8))
m = Basemap(projection= 'lcc', resolution=None,
           width=8E6, height=8E6,
           lat_0=44, lon_0=-93,)
m.etopo(scale=0.5, alpha=0.5)

# Map (long, lat) to (x, y) for plotting
x, y = m(-93.188986, 44.943439)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, 'UniversityofStThomas', fontsize=12);






Intitializing Scala interpreter ...

Spark Web UI available at http://10.80.102.113:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1543416756637)
SparkSession available as 'spark'


SyntaxError: ('invalid syntax', ('python cell', 1, 1, '%matplotlib inline\n'))